In [1]:
import requests as re
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
import datetime
import pprint

In [2]:
pd.set_option('display.max_colwidth', -1)

### Get Forums and Subforums
Do not need to run this section if have the list of subforums

In [ ]:
def getForums(site_url):
    col=['forum','subforum','forum_url']
    top_df = pd.DataFrame(columns=col)
    r = re.get(site_url)
    page = r.text
    soup = BeautifulSoup(page, 'html.parser')
    for element in soup.find_all('td', {"class": "alt1Active"}):
        forum_title_div = element.div
        forum_link = forum_title_div.a
        forum_url = site_url + forum_link['href']
        if('.com' in forum_link['href']):
            forum_url = forum_link['href']
        forum_title = forum_link.strong.string
        subforum = ''

        row = pd.DataFrame([[forum_title,subforum,forum_url]], columns=col)
        if(len(top_df)==0):
            top_df = row
        else:
            top_df = top_df.append(row, ignore_index=True) #df.append doesn't work inplace

        for child in element.find_all('div', {"style" : "margin-top:6px"}):
            if child.find('strong') != None: #has a Sub-Forums section
                for subforum_link in child.find_all('a'):
                    subforum = subforum_link.contents[0]
                    forum_url = site_url + subforum_link['href']
                    if('.com' in subforum_link['href']):
                        forum_url = subforum_link['href']
                    row = pd.DataFrame([[forum_title,subforum,forum_url]], columns=col)
                    if(len(top_df)==0):
                        top_df = row
                    else:
                        top_df = top_df.append(row, ignore_index=True) #df.append doesn't work inplace
    return top_df

In [ ]:
site_url = 'https://forums.hardwarezone.com.sg'
hwz = getForums(site_url)
print(len(hwz))
hwz

In [ ]:
hwz.to_csv('hwz_subforums.csv')

### Get Threads

In [3]:
def getThreads(forum_url):
    lastForumPage = False
    forum_cols = ['thread', 'datetime','thread_url']
    forum_df = pd.DataFrame(columns=forum_cols)
    forum_page_url = forum_url
    
    while(not lastForumPage):
        r2 = re.get(forum_page_url)
        forum_page = r2.text
        forum_page_soup = BeautifulSoup(forum_page, 'html.parser')

        if (forum_page_soup.find('a', text='Next ›') == None):
            lastForumPage = True
        else:
            forum_page_url = site_url + forum_page_soup.find('a', text='Next ›')['href']
        
        forum_num = subforum_url.strip('/').split('-')
        forum_num = forum_num[-1]
        
        thread_body = forum_page_soup.find('tbody',{'id': f'threadbits_forum_{forum_num}'})
        
        for thread in thread_body:
            if len(thread) == 1: 
                continue
            try: 

                # Retrieve Date 
                date_block = thread.find_all('td', class_ = "alt2")[1].find('div', class_= "smallfont" ).contents
                date = date_block[0].strip()
                if date == 'Yesterday' : 
                    date = (datetime.date.today() - datetime.timedelta(1)).strftime('%Y-%m-%d')
                elif date == 'Today': 
                    date = datetime.date.today().strftime('%Y-%m-%d')
                    
                if '2020' not in date and '2021' not in date: 
                    if 'class="hwz-sticky"' not in str(thread):
                        return forum_df   # Return forum_df with threads from 2020 onwards
                    else:
                        continue

                # Retrieve Thread Title
                title = thread.find('a', {"id": lambda x: x and 'thread_title_' in x}).contents[0]
                print(title)

                # Retrieve Time
                time = thread.find('span', class_ = 'time').text

                # Retrieve Date and Time
                date_time = date + ' ' + time

                # Retrieve Thread Url           
                url = site_url + thread.find('a', {"id": lambda x: x and 'thread_title_' in x})['href']        

                row = pd.DataFrame([[title, date_time, url]], columns=forum_cols)
                if(len(forum_df)==0):
                    forum_df = row
                else:
                    forum_df = forum_df.append(row, ignore_index=True) #df.append doesn't work inplace

            except:
                continue
                
    forum_df.drop_duplicates(inplace = True)
                
    return forum_df

In [ ]:
%%time
site_url = 'https://forums.hardwarezone.com.sg'

# manually key in subforum_url & subforum
subforum_url = 'https://forums.hardwarezone.com.sg/current-affairs-lounge-17/'
subforum = 'current-affairs-lounge'

# Change the directory
os.chdir('.')

threads = getThreads(subforum_url)

[BREAKING NEWS] Singapore upgraded travel advisory to deferring ALL TRAVEL aboard! :O
Covid19 - Dissenting views from around the world.
Loan Shark Harassment and how to stop it - Part 2
[香港世纪大乱局] How many will protest on Sunday? [Live Stream][现场直播] - Part 9
PAP government is in cahoots with rich developers to appropriate wealth capital for o
Doctors are actually the glorified salesmen of pharmaceutical company drugs
Many in Singapore work till they die.
News | Terence Loh resigns from Bellagraph Nova Group
Why is the SG Media So Against Trump?
I seriously advise true blue Singaporeans to migrate
Legal Clinic
Integrated Plans: Panel Doctors System; You are not a car
[LOTO] Pritam Singh: Publish Names of Recalcitrant Firms & Call For Anti-Discrimination Legislation
Illuminati New World Order
Instead of increasing GST, Freehold/ expensive residential property owners ought to p
China couple accused of obstructing COVID-19 containment work 'not averse' to pleading guilty: Lawye
Kong Hee is 

Analysis: Whither Wang Qishan? All eyes on Xi's right-hand man
(Breaking) China's experimental COVID-19 vaccine appears safe: Study
[成也川普，败也川普] Sick Trump scares Tiongkok
Why Europe is right to push back against China and Russia
What are the odds Trump will be readmitted to Hospital by the end of this Week
完了完了！Democrats amass a commanding lead in returned 4.2 million mail ballots.
To Japanese ambassador : Pl la....open up Japan for Singaporeans la for tourism
Trump Administration Announces Overhaul of H-1B Visa Program
3 weeks+ to USA election. Before you know it, it's over. Very fast
Why trump the face orange colour one
Trump says vote for me and you'll get a huge stimulus deal after the election
First German Invincible-class submarine to be delivered to Singapore Navy in 2022
[Breaking]Trump saved by sinkies siol!
[Holey Moley!] India developed a SMART Missile that launches torpedo more than 600km away
Antibodies in experimental treatment for President Trump from NCID, Singapore
Tru

F35 can fire missile from 200KM or longer but can hit meh???
Advice on handling online scam..
Breaking! Congressman intro bill to scrap One China Policy Resume Ties with Taiwan!
[LATEST] Wechat and Tiktok R.I.P
US defies world to say Iran UN sanctions back in effect
[GE2016 - A Challenge to All EDMWers] I AM NOT XENOPHOBIC, I JUST LOVE SINGAPORE & SINGAPOREANS MORE
[GPGT] Are you willing to donate $ to help this CECA FTrashes's medical bill in Singapore?
Breaking! US congressmen introduce Taiwan Invasion Prevention Act! No more grey area!
China returns back to Stone age with only 40nm chips. Can't manufacture smart phones!
Russia, China, Iran to hold massive joint military exercise
GVGT: US 155mm Paladin Howitzer now can shot down a cruise missiles!!
Breaking! US envoy to United Nation, meet Taiwan official in New York over lunch!
Australia eyes trade with India as China spat sours relationship
[美国噩梦] PRC combat pilot struck fear into USA by singlehandedly destroying 17 fighter jets 

st Forum: MAS request of banks not fair to retail investors
[GLGT] huawei going to uplorry?
America wants to Rob Again? Buy if cannot compete
The return of fake China milk powder... kym?
PSP is very concerned by the recent media reports on discriminatory hiring practices
China wants 'unpredictable' Trump to lose election
[BREAKING][GVGT] Just In!! Massive Explosion in Beirut, Lebanon!!!
Former Taiwanese diplomat and political commenters impression on CECA
Serious Implication - ST Article 7 Aug 20 -  "A bank executive recounted that the trading floor of t
Xiaomi, Oppo, Oneplus owners. U all scared? First hua wei, then tik tok now we chat BAN!!!
Why USA device can have backdoor but China device cannot have it?
Pompeo call Sg & Indo to take sides in SCS but both countries express no interest
beijing breaking news ! huawei beautiful win. 1 fist destroy usa chip industry.
Urumqi in China's Xinjiang offers free treatment to all confirmed, suspected and asym
GPGT: MPs for Sengkang GRC and Pri

How long drs take to approve
arm chio: CECA newly arrived Rafael fighter plane phua lok kok
[Forum] Workers' Party chief should be wary of involuntary exclusion
[SUNDAY TIMES] "I handed SG in very good shape to PM LHL, with VERY BEAUTIFUL ELECTION RESULTS..."
Soh Rui Yong and Gan Siow Huang are cousins
Who is Dr Tan See Leng?
GPGT: PSP reshuffled their party-leadership kym?
[GVGT] NTUC Sec Gen Ng, "As a Minister maybe I'll have more access to companies"
[You're Trumped] US Treasury must get substantial portion of  Tiktok sales fee
If your app is popular in America you must sell to America or force kena ban
GLGT: Intel accused of stealing China Patented FinFet technology
After tiktok next is DJI drone sell to America or ban
[prophecy] pap to lose pasir ris punggol grc next election
Delete pls
[Reddit] Chee Soon Juan has been Sec-Gen for 27 years without much success...
Is it valid to assume that WP will have lower chance to win East Coast now?
PAP Activist: "The old guard of the party r

PM Lee is almost a Master Strategist
Chee Soo Juan had garnered more than 40% of the votes for the first time in his political career
THARMAN MIST FONE CLEAN! WHY IVAN LIM FEATURED INSIDE THE JURONG GRC WINNING VIDEO?
[OFFICIAL - GE2020] MacPherson SMC discussions
How did George Yeo know he's losing the night before?
[Breaking] Sengkang GRC is The Workers Partys 🔨
Mr Ng Chee Meng from the PAP Sengkang team spoke to the media after they lost to Workers' Party
More than 10000 did not vote
GE2020 61.24% vs GE2011 60.1%, which one worse?
Cotton Chan still look good
oppo should stop complaining about GRC because WP abused GRC this time too..
Time for PAP to Fight the Virus, and Figure Out How to Repent
[OFFICIAL - GE2020] Holland-Bukit Timah GRC discussions
Hazel Poa Dr Chee is my choice for the 2 nmcp seats
[BREAKING] AGC advised that there is no offence made with regards to HSK's remarks
WILL THERE BE MASSIVE INFLUX OF NEW CITIZENS Next five years??
Which opposition party you think shoul

[GVGT] Dr Paul Tambyah reveals MOM's role in outbreak of COVID-19 within dormitories
[158th]Lawrence Wong:Doctors/experts requested MOM to advise employers not to send FW for testing!
[Night owls' Den] Awake? Care for a chat? - Part 56
(GOOD SIGN) Is The PAP Give Up Mulali At Bt Batok ?
MacPherson SMC Discussions
GPGT: The incumbent official statement on 10m.
NCMP scheme shows how hypocritical and dishonest the PAP government is.
[CNA] SDP's CSJ has not changed, cannot change, and will never change"
China passed a law for the whole world, non-Chinese critical of CCP can be arrested at HK airport
Dr Chee Soon Juan is blessed with a pretty wife & 2 gorgeous daughters
Will lim tean win over Josephine teo?
PAP's complacency, economy shutdown, GDP shrunk, recession sinkies lose jobs
Singapore economy finished if incapable people run the Government
How PAP devised the GRC system of elections to checkmate Singapore citizens and hold
China warns UK of 'consequences' over citizenship offer to 

4 students and 1 school employee contract COVID-19; all have 'low viral loads': MOE
[Breaking] Companies Burning Land Sued by Volunteering Grp
[Breaking] US says Tiongkok Hacking & Stealing its Wuhan Virus Vaccine Research!
How cum Govt don't nationalize the Dorms and pay operators to run them like the Buse Routes?
"The CCP's slaughter of unarmed Chinese civilians a tragedy that'll not be forgotten": White House
I'd rather USA dominate the world rather than China
Internet SG
Minister Wong want us to forget abt their past mishandling of CV-19, and LETS MOVE ON....LOL
[ANARCHY] UNITED STATES OF AMERICA REPORTS CURFEWS IN 25 CITIES. 21 YEAR OLD SHOT BY POLICE.
East Asians told to f**k off as racism becomes common in the UK
GPGT - This is what HK cockroaches say about Singapore/Sporeans
Three million HK residents eligible for UK citizenships!
Huawei sourcing mobile chips from MediaTek and UNISOC to beat U.S trade ban
It’s hair-raising and spine-chilling that our Trade and Industry Minister

UnicodeEncodeError: 'utf-8' codec can't encode characters in position 0-1: surrogates not allowed

Exception ignored in: 'pandas._libs.tslibs.util.get_c_string_buf_and_size'
UnicodeEncodeError: 'utf-8' codec can't encode characters in position 0-1: surrogates not allowed


In [ ]:
threads.count()

In [ ]:
threads.to_csv(subforum, encoding = 'utf-8', index = False)

### Get Posts - all pages

In [3]:
def getPosts(thread_url):
    #print(thread_url)
    lastThreadPage = False
    thread_cols = ['post_id','timestamp', 'post_text']
    thread_df = pd.DataFrame(columns=thread_cols)
    thread_page_url = thread_url
    
    count = 0
    while(not lastThreadPage):
        #print(thread_page_url)
        r3 = re.get(thread_page_url)
        thread_page = r3.text
        thread_page_soup = BeautifulSoup(thread_page, 'html.parser')

        if (thread_page_soup.find('a', text='Next ›') == None):
            lastThreadPage = True
        else:
            thread_page_url = site_url + thread_page_soup.find('a', text='Next ›')['href']

        thread_page_posts = thread_page_soup.find('div', {'id': 'posts'})
                
        try: 
            for post in thread_page_posts.find_all('div', {'class': 'post-wrapper'}):
                
                # Retrieve Date & Time
                datetime_raw = post.find('a', {'name': lambda x: x and x.find('post') == 0}).nextSibling.strip()
                date = datetime_raw.split(',')[0]
                    
                if date == 'Yesterday' : 
                    date = (datetime.date.today() - datetime.timedelta(1)).strftime('%Y-%m-%d')
                elif date == 'Today': 
                    date = datetime.date.today().strftime('%Y-%m-%d')
                
                if '2020' not in date and '2021' not in date:
                    return thread_df
                    
                time = datetime_raw.split(',')[1]
                date_time = date + ' ' + time
                
                # Retrieve Post Text
                post_text = ""
                try:
                    post_text = post.find('div', {'class': 'post_message'}).get_text(' ', strip=True)
                except AttributeError as e: 
                    pass
                
                # Retrieve Post ID
                post_id = int(post.find('a', {'id': lambda x: x and 'postcount' in x, 'target': 'new'})['id'].lstrip('postcount'))
                
                row = pd.DataFrame([[ post_id, date_time, post_text]], columns=thread_cols)
                if(len(thread_df)==0):
                    thread_df = row
                else:
                    thread_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
        except:
            row = pd.DataFrame([[np.nan, np.nan, ""]], columns=thread_cols) #posts missing, thread may have been deleted
            if(len(thread_df)==0):
                thread_df = row
            else:
                thread_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
        
    thread_df['post_text'] = thread_df['post_text'].map(lambda x: x.encode('unicode-escape').decode('utf-8'))
    
    thread_df.drop_duplicates(inplace = True)

    return thread_df

#### Combine posts with threads

In [24]:
# Returns posts for each thread in specified subforum and policy 

def post_threads(df):
    new_df = pd.DataFrame(columns = ['thread', 'datetime','thread_url', 'post_id','post_timestamp', 'post_text'])
    df_col = new_df.columns

    for index in df.index:
        try:
            thread_url = df['thread_url'][index]
            posts = getPosts(thread_url)
            for p_index in posts.index:
                row = pd.DataFrame([[df['thread'][index], df['datetime'][index], df['thread_url'][index], 
                                    posts['post_id'][p_index], posts['timestamp'][p_index], posts['post_text'][p_index]]], 
                                   columns = df_col)
                new_df = new_df.append(row, ignore_index = True, sort = False)
        except:
            continue
        
    return new_df

In [26]:
cwd = os.path.abspath('.')

# Change the directory to the respective directory
os.chdir(r"./Threads/../")

files = os.listdir(cwd)
files

['Done',
 'get_threads.py',
 'hwz_vaccination_threads.csv',
 'old',
 'Posts',
 'to check']

In [27]:
'''Create a folder named "Posts" in the current directory'''

site_url = 'https://forums.hardwarezone.com.sg'

for file in files:
    if file.endswith('.csv'):
        df = pd.read_csv(f'{file}')
        new_df = post_threads(df)
        filename = f'{file}'[:-11] + 'posts'
        filepath = os.path.join('Posts', filename + '.csv')
        new_df.to_csv(filepath, index = False)
        print(f'Complete: {file}')

Complete: hwz_vaccination_threads.csv
